On commence par importer les différentes librairies nécessaires

In [1]:
import pandas as pd
import time
import shapely
import shapely.geometry
from shapely.strtree import STRtree
import shapefile
from pyproj import Proj, transform, CRS

On importe les bases de données SIRENE

In [2]:
folder="/mnt/c/Users/guera/projects/E-CUBE" #Attention à bien préciser le nom de dossier correspondant au dépôt git

In [2]:
siren_eta_geo = pd.read_csv(folder+"/data/SIREN/StockEtablissementActif_utf8_geo.csv", sep=',',usecols=["siren", "siret", "codeCommuneEtablissement", "longitude", "latitude"])
siren_ul = pd.read_csv(folder+"/data/SIREN/StockUniteLegale_utf8.csv", sep=',',usecols=["siren", "categorieJuridiqueUniteLegale"])

/home/guera/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On importe les contours des IRIS de l'IGN pour la France Métropolitaine et les DROM

In [3]:
file_name_metro="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019/CONTOURS-IRIS.shp"
sh_metro = shapefile.Reader(file_name_metro)
shapes_metro= sh_metro.shapes()

file_name_guad="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGAF09UTM20_GLP-2019/CONTOURS-IRIS.shp"
sh_guad = shapefile.Reader(file_name_guad)
shapes_guad= sh_guad.shapes()

file_name_mart="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGAF09UTM20_MTQ-2019/CONTOURS-IRIS.shp"
sh_mart = shapefile.Reader(file_name_mart)
shapes_mart= sh_mart.shapes()

file_name_may="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGM04UTM38S_MYT-2019/CONTOURS-IRIS.shp"
sh_may = shapefile.Reader(file_name_may)
shapes_may= sh_may.shapes()

file_name_reu="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGR92UTM40S_REU-2019/CONTOURS-IRIS.shp"
sh_reu = shapefile.Reader(file_name_reu)
shapes_reu= sh_reu.shapes()

file_name_guy="/mnt/c/Users/guera/projects/E-CUBE/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_UTM22RGFG95_GUF-2019/CONTOURS-IRIS.shp"
sh_guy = shapefile.Reader(file_name_guy)
shapes_guy= sh_guy.shapes()

On récupère dans des DataFrames de pandas les correspondances entre contours d'IRIS et informations sur l'IRIS

In [4]:
from simpledbf import Dbf5
iris_metro_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019/CONTOURS-IRIS.dbf")
iris_metro_df=iris_metro_dbf.to_dataframe()

iris_guad_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGAF09UTM20_GLP-2019/CONTOURS-IRIS.dbf")
iris_guad_df=iris_guad_dbf.to_dataframe()

iris_mart_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGAF09UTM20_MTQ-2019/CONTOURS-IRIS.dbf")
iris_mart_df=iris_mart_dbf.to_dataframe()

iris_may_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGM04UTM38S_MYT-2019/CONTOURS-IRIS.dbf")
iris_may_df=iris_may_dbf.to_dataframe()

iris_reu_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_RGR92UTM40S_REU-2019/CONTOURS-IRIS.dbf")
iris_reu_df=iris_reu_dbf.to_dataframe()

iris_guy_dbf=Dbf5(folder+"/data/CONTOURS-IRIS_2-1__SHP__FRA_2020-01-01/CONTOURS-IRIS/1_DONNEES_LIVRAISON_2020-01-00139/CONTOURS-IRIS_2-1_SHP_UTM22RGFG95_GUF-2019/CONTOURS-IRIS.dbf")
iris_guy_df=iris_guy_dbf.to_dataframe()

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


On fait un premier tri pour supprimer les personnes physiques

In [3]:
siren = siren_eta_geo.merge(siren_ul)
siren_without_auto = siren.loc[siren.loc[:, "categorieJuridiqueUniteLegale"] != 1000]

NameError: name 'siren_eta_geo' is not defined

In [4]:
siren_without_auto_iris = pd.read_csv(folder+"/data/SIREN/StockEtablissementActif_utf8_geo_iris.csv")

/home/guera/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (12,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On récupère avec la librairie pyproj les projections utilisées par l'IGN pour les différentes régions

In [6]:
#Lambert 93 -> France Metropolitaine
crs_L93 = CRS.from_epsg(2154)
L93_Proj = Proj(crs_L93)

#WGS -> coordonnées longitude, latitude
crs_WGS = CRS.from_epsg(4326)
WGS_Proj = Proj(crs_WGS)

#RGAF09UTM20 -> Antilles Françaises (Guadeloupe, Martinique)
crs_RGAF09UTM20 = CRS.from_epsg(5490)
RGAF09UTM20_Proj = Proj(crs_RGAF09UTM20)

#RGFG95UTM22 -> Guyane française
crs_RGFG95UTM22 = CRS.from_epsg(2972)
RGFG95UTM22_Proj = Proj(crs_RGFG95UTM22)

#RGR92UTM40S -> Réunion
crs_RGR92UTM40S = CRS.from_epsg(2975)
RGR92UTM40S_Proj = Proj(crs_RGR92UTM40S)

#RGM04UTM38S -> Mayotte
crs_RGM04UTM38S = CRS.from_epsg(4471)
RGM04UTM38S_Proj = Proj(crs_RGM04UTM38S)

On crée un DataFrame **communes** qui comprend la liste de toutes les IRIS avec leur contour et leur commune associée

In [7]:
listcc=[]
listshape=[]
listiris=[]

for i in range(len(shapes_metro)):
    cc=iris_metro_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_metro[i].points)
    iris=iris_metro_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)
    
for i in range(len(shapes_guad)):
    cc=iris_guad_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_guad[i].points)
    iris=iris_guad_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)

for i in range(len(shapes_mart)):
    cc=iris_mart_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_mart[i].points)
    iris=iris_mart_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)
    
for i in range(len(shapes_may)):
    cc=iris_may_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_may[i].points)
    iris=iris_may_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)
    
for i in range(len(shapes_reu)):
    cc=iris_reu_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_reu[i].points)
    iris=iris_reu_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)
    
for i in range(len(shapes_guy)):
    cc=iris_guy_df.loc[i,"INSEE_COM"]
    poly=shapely.geometry.Polygon(shapes_guy[i].points)
    iris=iris_guy_df.loc[i,"CODE_IRIS"]
    listcc.append(cc)
    listshape.append(poly)
    listiris.append(iris)

communes=pd.DataFrame({'code commune':listcc, 'contour iris':listshape, 'iris':listiris})
communes

,code commune,contour iris,iris
0,72191,"POLYGON ((497887.4 6747662.4, 497907.2 6747658...",721910000
1,77248,"POLYGON ((685757.7 6868592.3, 685832.5 6868319...",772480000
2,51426,"POLYGON ((757292.9 6847598.4, 757334.3 6847081...",514260000
3,81199,"POLYGON ((646146.9 6328153.2, 646124.9 6328229...",811990000
4,59225,"POLYGON ((767596 7022269.1, 767242.2 7022129.4...",592250102
...,...,...,...
49399,97307,"POLYGON ((351503 534810.5, 351509.2 534806.5, ...",973070105
49400,97311,"POLYGON ((164025.4 609416.5, 164105.7 609443.7...",973110101
49401,97307,"POLYGON ((348751.6 542027.1, 348758.9 542017.3...",973070108
49402,97362,"POLYGON ((130962.6 457860.7, 131036.1 457823.7...",973620000


En s'aidant du DataFrame précedent, on crée un dictionnaire avec pour clés les codes communes et pour valeurs un arbre de recherche avec les IRIS de la commune et un lexique (autre dictionnaire) permettant d'associer à chaque forme de contour le numéro d'IRIS 

In [8]:
d={}
for i in range(len(communes)):
    try:
        (p,l)=d[communes.iloc[i,0]]
    except:
        p,l=[],[]
    poly = communes.iloc[i,1]
    iris=communes.iloc[i,2]
    p.append(poly)
    l.append(iris)
    d[communes.iloc[i,0]] = (p,l)

for i in d:
    p,l=d[i]    
    t=STRtree(p)
    lex=dict((id(p[i]),l[i]) for i in range(len(p)))
    d[i]=(lex,t)

On obtient finalement la fonction qui permet de donner l'IRIS pour la ième ligne de la base SIRENE

In [9]:
def iris_commune_dict(i):
    x2,y2=0,0
    cc=str(siren_without_auto.loc[i,"codeCommuneEtablissement"])
    if len(cc)==4:
        cc='0'+cc
    try:
        lex,tree=d[cc]
    except:
        return None
    if len(lex) == 1:
        return list(lex.values())[0]
    else:
        lat=siren_without_auto.loc[i,"latitude"]
        lon=siren_without_auto.loc[i,"longitude"]
        if cc[:2]=='97':
            if cc[2]=='1':
                x2,y2 = transform(WGS_Proj,RGAF09UTM20_Proj,lat,lon)
            elif cc[2]=='2':
                x2,y2 = transform(WGS_Proj,RGAF09UTM20_Proj,lat,lon)
            elif cc[2]=='3':
                x2,y2 = transform(WGS_Proj,RGFG95UTM22_Proj,lat,lon)
            elif cc[2]=='4':
                x2,y2 = transform(WGS_Proj,RGR92UTM40S_Proj,lat,lon)
            elif cc[2]=='5':
                x2,y2 = transform(WGS_Proj,RGM04UTM38S_Proj,lat,lon)
        else:
            x2,y2 = transform(WGS_Proj,L93_Proj,lat,lon)
        query_geom=shapely.geometry.Point(x2,y2)
        ir=tree.query(query_geom)
        if ir==[]:
            return None
        return lex[id(ir[0])]

On écrit pour finir la fonction qui va effecuter à l'aide d'une boucle les calculs et les enregistrer toutes les 10000 valeurs

In [10]:
def save(a,nb):
    global siren_without_auto
    global siren_without_auto_iris
    l=len(siren_without_auto)
    for j in range(nb):
        t0=time.time()
        df=siren_without_auto.iloc[(a + 10000*j):(a + 10000*(j+1)),:].copy()
        df['IRIS']=0
        for i in range(10000):
            if (a + 10000*j + i) == l:
                break
            df.iloc[i,6]=iris_commune_dict(siren_without_auto.index[a + 10000*j + i])
        t1=time.time()
        print(t1-t0)
        print(j+1)
        siren_without_auto_iris=pd.concat([siren_without_auto_iris,df], axis=0)
        siren_without_auto_iris.to_csv(folder+"/data/SIREN/StockEtablissementActif_utf8_geo_iris.csv")

In [14]:
siren_without_auto_iris.head(7000000)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,siren,siret,codeCommuneEtablissement,longitude,latitude,categorieJuridiqueUniteLegale,IRIS
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15850944,1585094400024,1376,4.989048,45.823935,5720,013760000
1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,15850944,1585094400016,21295,4.992864,47.228989,5720,212950000
2,2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,15851793,1585179300479,1134,4.904811,46.282463,5699,011340000
3,3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,15851793,1585179300743,1057,4.913975,46.407388,5699,010570000
4,4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,15851793,1585179300917,1053,5.228366,46.220760,5699,010530604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999995,6999995,11230966.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514418763,51441876300017,95504,2.291358,49.104907,6540,9.5504e+08
6999996,6999996,11230967.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514420728,51442072800016,95018,2.225208,48.943163,5499,9.5018e+08
6999997,6999997,11230968.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514422021,51442202100014,95476,2.061672,49.060093,6540,9.5476e+08
6999998,6999998,11230969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514422658,51442265800013,95585,2.371295,48.977743,5499,9.5585e+08


In [12]:
save(7050000,35)

1420.8633630275726
1
1031.7480905056
2
1024.495144367218
3
1036.3973021507263
4
1041.5164940357208
5
1048.0127561092377
6
951.5996894836426
7
928.3935422897339
8
952.1553509235382
9
951.6559336185455
10
948.6928040981293
11
972.3161528110504
12
1014.7483875751495
13
1638.32386136055
14
1671.6030094623566
15
1082.321933746338
16
1057.3812246322632
17
1042.5089287757874
18
1050.8106200695038
19
1053.5083937644958
20
1045.9765975475311
21
1047.7165999412537
22
616.1101586818695
23
5.942477464675903
24
6.237951993942261
25
5.636692523956299
26
5.74652886390686
27
7.543617010116577
28
5.342889785766602
29
5.785607576370239
30
5.603188514709473
31
5.761425018310547
32
5.461548805236816
33
1.970080852508545
34


IndexError: index 7390000 is out of bounds for axis 0 with size 7383430

In [31]:
df_iris=siren_without_auto_iris[['siret','IRIS']]
df_iris

,siret,IRIS
0,1585094400024,013760000
1,1585094400016,212950000
2,1585179300479,011340000
3,1585179300743,010570000
4,1585179300917,010530604
...,...,...
7383425,88270263200010,NaN
7383426,94625069300029,NaN
7383427,95000757500018,NaN
7383428,95005160700014,NaN


In [32]:
df_iris = df_iris.astype(str)
df_iris.head(7000000)

,siret,IRIS
0,1585094400024,013760000
1,1585094400016,212950000
2,1585179300479,011340000
3,1585179300743,010570000
4,1585179300917,010530604
...,...,...
6999995,51441876300017,955040000.0
6999996,51442072800016,950180402.0
6999997,51442202100014,954760105.0
6999998,51442265800013,955850125.0


In [34]:
df_iris.to_csv(folder+"/data/SIREN/StockEtablissementActif_utf8_geo_iris_propre.csv",index=False)